In [1]:
#installing ollama
#!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [3]:
#pulling llama3 using ollama
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 943 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  38 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  64 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 118 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 138 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 177 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 209 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 241 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 250 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   6% ▕▏ 289 MB/4.7 GB  289 MB/s     15spulling manifest 
pulling 6a0746a

In [4]:
#testing llama 3 is availble
!ollama list

NAME             ID              SIZE      MODIFIED               
llama3:latest    365c0bd3c000    4.7 GB    Less than a second ago    


In [5]:
!pip install langchain_community langchain langchain-openai langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube python-dotenv tiktoken pinecone-client scikit-learn ruff pypdf faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to pr

In [6]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

In [7]:

#setting up model dynamically
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama3"

#VECTOR_STORE = "default"
VECTOR_STORE = "FAISS"
#VECTOR_STORE = "pinecone"

In [8]:
#loading model based on selection
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

#testing model invoke
model.invoke("what is ML")

"ML stands for Machine Learning. It's a subfield of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed.\n\nMachine learning enables computers to improve their performance on a task over time, based on the data they receive. This is in contrast to traditional programming, where the algorithm's behavior is determined by human-designed rules and logic.\n\nSome key concepts in machine learning include:\n\n1. **Training**: The process of feeding an algorithm a dataset and letting it learn from the patterns and relationships within that data.\n2. **Model**: A mathematical representation of the relationships learned during training, which can be used to make predictions or decisions.\n3. **Prediction**: The output produced by a trained model when given new, unseen data.\n4. **Supervised learning**: When a model is trained on labeled data (i.e., data with pre-defined outputs) and learns to pred

In [9]:
#creating parser
parser = StrOutputParser()

chain = model | parser

#testing model invoke with parser
chain.invoke("what is ML")

"ML can refer to several things, depending on the context:\n\n1. **Machine Learning**: Machine learning (ML) is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. The algorithms learn by finding patterns in the data, making them useful for tasks such as image recognition, natural language processing, and predictive modeling.\n2. **Microsoft Loop** (ML): Microsoft Loop is a cloud-based collaboration platform that allows users to create loops of connected content, enabling seamless sharing and updating of information across teams.\n3. **Main Line** (ML): In railway terminology, the main line refers to the primary route or track used by trains for regular service.\n4. **Medical Laboratory** (ML): A medical laboratory is a facility where clinical testing and analysis are performed to diagnose and manage diseases.\n\nIn general, ML (Machine Learning) has become a popular buzzword i

In [10]:
#creating template
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

#testing prompt
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t\nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [11]:
#loading pdf files
import copy

loader = PyPDFLoader("/content/gxocompany.pdf")
pages = loader.load_and_split()
pages_original = copy.deepcopy(pages)
pages_original[35]

Document(metadata={'source': '/content/gxocompany.pdf', 'page': 24}, page_content='Basis of Presentation\nOn August 2, 2021, the Company became a standalone publicly traded company and its financial statements post the separation from XPO, Inc. (the “Separation”) are now prepared on a consolidatedbasis. Before the Separation, GXO’s historical financial statements were prepared on a standalone combined basis and were derived from the Consolidated Financial Statements and accountingrecords of XPO. The combined Consolidated Financial Statements for all periods presented before the Separation are now referred to as “Consolidated Financial Statements” and have been preparedunder the U.S. generally accepted accounting principles (“GAAP”). GXO has one reportable segment.\nOn October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisitio

In [12]:
# Define the page numbers you want to extract
target_page_numbers = [35,36]  # Example page numbers

# Create an empty string to store the combined content
combined_content = ""

# Iterate through the pages and extract content from the target pages
page_number = 0 # Start with page number 1
for page in pages_original:
    if page_number in target_page_numbers:
        combined_content += page.page_content
        # You can add a separator (e.g., "\n\n") between page contents if needed
    page_number += 1 # Increment the page number for the next page

# Print or use the combined content
print(combined_content)

Basis of Presentation
On August 2, 2021, the Company became a standalone publicly traded company and its financial statements post the separation from XPO, Inc. (the “Separation”) are now prepared on a consolidatedbasis. Before the Separation, GXO’s historical financial statements were prepared on a standalone combined basis and were derived from the Consolidated Financial Statements and accountingrecords of XPO. The combined Consolidated Financial Statements for all periods presented before the Separation are now referred to as “Consolidated Financial Statements” and have been preparedunder the U.S. generally accepted accounting principles (“GAAP”). GXO has one reportable segment.
On October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisition of Clipper Logistics plc (“Clipper”), an omnichannel retail logistics specialist ba

In [13]:
import re
# Clean unnecessary data in pages
for page in pages:
    # Remove headers and footers (example patterns)
    page.page_content = re.sub(r"Header text pattern", "", page.page_content)
    page.page_content = re.sub(r"Footer text pattern", "", page.page_content)

    # Remove specific patterns (example)
    page.page_content = re.sub(r"© \d{4} Company Name", "", page.page_content)

    # Remove page numbers (example)
    page.page_content = re.sub(r"\d+/\d+", "", page.page_content)

    # Remove special characters (except for alphanumeric, spaces, $, ., and ,)
    page.page_content = re.sub(r"[^a-zA-Z0-9 \$,.]", "", page.page_content) # Added , and . to the excluded characters

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on document size
    chunk_overlap=200,  # Ensures important context isn't lost
    separators=["\n\n", "\n", " "]  # Prioritize splitting at paragraph breaks
)

#chunks = text_splitter.split_documents(pages)
chunks = text_splitter.create_documents([combined_content])

In [15]:
#loading vector store based on selection
if VECTOR_STORE == "FAISS":
    #vectorstore = FAISS.from_documents(pages, embeddings)
    vectorstore = FAISS.from_documents(chunks, embeddings)
else:
    vectorstore = DocArrayInMemorySearch.from_documents(chunks, embedding=embeddings, verbose=True)

In [16]:
#setting up retriver
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

#testing retriver
retriever.invoke("balance sheets")

[Document(page_content='Transaction and integration costs were $34 million in 2023, compared with $61 million for 2022. Transaction and integration costs in 2023 included $20 million related to the integration of Clipperand $12 million related to the PFS Acquisition, reflecting costs associated with advisory fees and severance costs. Transaction and integration costs in 2022 primarily related to $46 million related tothe Clipper Acquisition, reflecting costs associated with financing arrangements, advisory fees and integration costs, and $15 million from the Separation, primarily reflecting rebranding costs.'),
 Document(page_content='On October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisition of Clipper Logistics plc (“Clipper”), an omnichannel retail logistics specialist based in Leeds, England (the “Clipper Acquisition”

In [17]:
#all_text = ""
#for page in pages:
#    all_text += page.page_content

In [18]:
def print_context(inputs):
  print("Context:")
  print(inputs["context"])
  return inputs

In [19]:
#chaining all operations
from langchain_core.runnables import RunnableLambda

chain = (
    {
        "context": itemgetter("question") | retriever, #| RunnableLambda(lambda x: all_text),
        #"context": RunnableLambda(lambda x: combined_content),
        "question": itemgetter("question"),
    }
    | RunnableLambda(print_context)
    | prompt
    | model
    | parser
)

In [21]:
#posting questions
questions = [
    #"provide finanacial analysis as underwriter in financial company by going to complete context that has in company annual reports",
     #"how is the revenue growth of company in 2023",
   """
   Analyze the financial statements and provide a structured summary of Revenue growth for FY23.
Your response should include:
1. Total revenue for FY23 and FY22.
2. Year-over-year (y/y) change in absolute ($) and percentage (%) terms.
3. Key factors contributing to the revenue change, such as:
   - Organic growth (operational expansion)
   - Revenue from acquired entities
   - Foreign currency impact
   """,
    """
Analyze the financial statements for FY23, focusing on the relationship between Direct Operating Expenses and Cost of Goods Sold (COGS).

Specifically, provide a detailed analysis, assuming Direct Operating Expenses are a significant component of COGS:

1. What was the total Direct Operating Expenses for FY23 and FY22?
2. Based on this, estimate the total COGS for FY23 and FY22.
3. Calculate the year-over-year (y/y) change in estimated COGS and Gross Profit (GP), both in absolute dollars (MM) and percentage (%).
4. Provide a breakdown of the key components of Direct Operating Expenses, such as personnel costs, rent expenses, and transportation costs.
5. Explain the factors that contributed to the changes in Direct Operating Expenses (and thus estimated COGS) and Gross Profit, such as acquisitions, changes in expenses, or other relevant factors.
6. Discuss the impact of these changes on Gross Profit Margin.

Ensure the response is structured and includes specific numerical values where applicable.
""",
    """
Analyze the financial statements for FY23, focusing on transactions and integration costs.

Specifically, provide the following information:

1. The total amount of transactions and integration costs for FY23 and FY22.
2. The year-over-year (y/y) change in transactions and integration costs, both in absolute dollars (MM) and percentage (%).
3. A breakdown of the factors contributing to the change in transactions and integration costs, including any specific acquisitions or events that impacted these costs.

Ensure the response is structured and includes specific numerical values where applicable.
""",
    """
Analyze the financial statements for FY23, focusing on operating expenses and net operating profit.

Specifically, provide the following information:

1. The total amount of operating expenses for FY23 and FY22, along with the year-over-year (y/y) change in absolute dollars (MM) and percentage (%).
2. Identify the primary factors contributing to the change in operating expenses, such as specific expense categories.
3. Analyze operating expenses as a percentage of revenue for FY23 and FY22, highlighting any significant changes or trends.
4. Provide the net operating profit for FY23 and FY22, along with the y/y change in absolute dollars (MM) and percentage (%).
5. Discuss the overall relationship between operating expenses, revenue, and net operating profit for FY23.

Ensure the response is structured, includes specific numerical values, and provides clear explanations for the observed changes.
"""

]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: 
   Analyze the financial statements and provide a structured summary of Revenue growth for FY23.
Your response should include:
1. Total revenue for FY23 and FY22.
2. Year-over-year (y/y) change in absolute ($) and percentage (%) terms.
3. Key factors contributing to the revenue change, such as:
   - Organic growth (operational expansion)
   - Revenue from acquired entities
   - Foreign currency impact
   
Context:
[Document(page_content='Transaction and integration costs were $34 million in 2023, compared with $61 million for 2022. Transaction and integration costs in 2023 included $20 million related to the integration of Clipperand $12 million related to the PFS Acquisition, reflecting costs associated with advisory fees and severance costs. Transaction and integration costs in 2022 primarily related to $46 million related tothe Clipper Acquisition, reflecting costs associated with financing arrangements, advisory fees and integration costs, and $15 million from the Separa